In [1]:
%pip install boto3


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import re
import base64
import boto3

# Constants
GITHUB_API_URL = "https://api.github.com/search/code"
QUERY = "NODE_CLASS_MAPPINGS language:Python isfork"
TOKEN = "github_api_key"  # Replace with your GitHub token
DYNAMODB_TABLE_NAME = "NodePackage-u66pcvc7szb5tjn7j44exgi2gq-prod"
AWS_ACCESS_KEY = "xxx"  # Replace with your AWS access key
AWS_SECRET_KEY = "xxxxxxx"  # Replace with your AWS secret key
# Setting up the GitHub headers
headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json',
}

# Setting up the DynamoDB client
dynamodb = boto3.resource('dynamodb',
                          region_name='us-west-1',
                          aws_access_key_id=AWS_ACCESS_KEY,
                          aws_secret_access_key=AWS_SECRET_KEY)
table = dynamodb.Table(DYNAMODB_TABLE_NAME)

# Function to parse node mappings from code
def parse_node_mappings(code):
    # Regular expression to match the entire NODE_CLASS_MAPPINGS dictionary
    dict_pattern = r"NODE_CLASS_MAPPINGS\s*=\s*\{([^}]+)\}"
    dict_match = re.search(dict_pattern, code, re.DOTALL)

    if dict_match:
        # Extract the dictionary content
        dict_content = dict_match.group(1)

        # Regular expression to extract each node_name: node_class pair
        pair_pattern = r'"(\w+)":\s*(\w+)'
        return re.findall(pair_pattern, dict_content)
    else:
        return []

# Function to save node to DynamoDB
def save_to_dynamodb(node_name, node_class, git_html_url, file_html_url, author_name, author_avatar_url, description):
    try:
        table.put_item(
            Item={
                'id': node_name.replace(" ", "_"),
                'nodeClassName': node_class,
                'gitHtmlUrl': git_html_url,
                'fileHtmlUrl': file_html_url,  # Adding the file URL
                'authorName': author_name,
                'authorAvatarUrl': author_avatar_url,
                'description': description,
                'totalInstalls': 1
            }
        )
        return True
    except Exception as e:
        print(f"Error saving {node_name} to DynamoDB: {e}")
        return False

# Function to process each file from search results
def process_files(files, file_counter, mapping_counter):
    for file in files:
        file_counter += 1
        content_url = file['git_url']
        file_html_url = file['html_url']  # GitHub file URL
        repo_html_url = file['repository']['html_url']
        author_name = file['repository']['owner']['login']
        author_avatar_url = file['repository']['owner']['avatar_url']
        description = file['repository']['description']

        # Getting the file content
        response = requests.get(content_url, headers=headers)
        if response.status_code != 200:
            continue
        content = response.json().get('content')
        if not content:
            continue

        # Decoding the base64 content
        decoded_content = base64.b64decode(content).decode('utf-8')

        # Parsing node mappings
        node_mappings = parse_node_mappings(decoded_content)
        for node_name, node_class in node_mappings:
            mapping_counter += 1

            # Print each node mapping
            print(f"File #{file_counter}, Mapping #{node_name}:{node_class}")

            # Save each node to DynamoDB and handle errors
            if not save_to_dynamodb(node_name, node_class, repo_html_url, file_html_url, author_name, author_avatar_url, description):
                print("Failed to save to DynamoDB. Continuing with next mapping.")
    return file_counter, mapping_counter

# Main script to make the request and process results
def main():
    page = 1
    file_counter = 0
    mapping_counter = 0
    while True:
        params = {'q': QUERY, 'page': page, 'per_page': 100}
        response = requests.get(GITHUB_API_URL, headers=headers, params=params)
        print(response.json())
        if response.status_code != 200:
          print(response.status_code)
          break
        search_results = response.json()
        files = search_results.get('items', [])
        if not files:
          break
        file_counter, mapping_counter = process_files(files, file_counter, mapping_counter)
        print(f"Completed page {page}. Total files processed: {file_counter}, Total mappings processed: {mapping_counter}")
        page += 1

# Run the script
main()


{'message': 'Bad credentials', 'documentation_url': 'https://docs.github.com/rest'}
401
